In [5]:
import torch
from torch.optim import Adam
from torch.nn import Linear, MSELoss,Sigmoid,BCELoss, Sequential,Module, Softmax,CrossEntropyLoss,ReLU, Conv2d, MaxPool2d
# conv2d와 maxpool2d를 추가한다.
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.fashion_mnist import load_data
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
# from tensorflow.keras.datasets.mnist import load_data
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [51]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
if USE_CUDA :
    DEVICE = torch.device('cuda')
else :
    DEVICE = torch.device('cpu')

In [ ]:
(x_train,y_train),(x_t,y_t) = load_data()
# /255로 정규화 진행
x = torch.FloatTensor( x_train.reshape( 60000, 1, 28, 28)/255 )
y = torch.LongTensor( y_train) # 다중분류
x_test = torch.FloatTensor( x_t.reshape( 10000, 1, 28, 28)/255 )
y_test = torch.LongTensor( y_t)

In [40]:
x_train, x_val, y_train, y_val = train_test_split( x,y,
                        test_size=0.2, stratify=y) 

In [61]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=1600, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1600, shuffle=True)

In [ ]:
print(x_train.shape)
print(x_val.shape)

torch.Size([48000, 1, 28, 28])
torch.Size([12000, 1, 28, 28])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module) :
    def __init__(self) :
        super(Net, self).__init__()
        # 컬러 사진이기 때문에 in_channels는 3을 준다.
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3,
                                padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,
                                padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,
                                padding=1)
        
        example_img = torch.zeros(1,1,28,28)
        flatten_size = self.get_flatten_size(example_img)
        print(flatten_size)
        
        self.fc1 = nn.Linear(flatten_size, out_features=36)
        self.fc2 = nn.Linear(in_features=36, out_features=10)
        self.smax = Softmax( dim=1)

    def get_flatten_size(self, x):
        x = F.relu( self.conv1(x) )
        x = F.relu( self.pool(x) )
        x = F.relu( self.conv2(x) )
        x = F.relu( self.conv3(x) )
        x = torch.flatten(x,1)
        return x.shape[1]
    
    def forward(self, x) :
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)

        # x = x.view(-1, 4096)
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5)
        x = self.fc2(x)

        return x

In [96]:
model = Net().to(DEVICE)
loss_fn = CrossEntropyLoss()
optimizer = Adam( model.parameters(), lr=0.01)
patience = 10
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(20):
    train_loss= 0
    
    model.train()
    for i, (batch_x, batch_y) in enumerate(train_loader):
        print( i, 'train')
        batch_x = batch_x.to(DEVICE)
        batch_y = batch_y.to(DEVICE)
        
        optimizer.zero_grad()
        hx = model( batch_x )
        cost = loss_fn(hx, batch_y)
        cost.backward()
        optimizer.step()
        # TRAIN 완료
        train_loss = train_loss + cost.item()
    train_loss = train_loss/len( train_loader) # 평균 학습 손실계산
    print('train loss is ', train_loss)
    
    model.eval() 
    val_loss = 0
    correct = 0
    
    with torch.no_grad() :
        for i, (batch_x, batch_y) in enumerate(val_loader) :
            print( i, 'evaluate')
            batch_x = batch_x.to(DEVICE)
            batch_y = batch_y.to(DEVICE)
            hx = model(batch_x)
            val_loss += F.cross_entropy(hx, batch_y, reduction='sum').item()
            
            pred = hx.argmax(1, keepdim=True)
            correct += pred.eq(batch_y.view_as(pred)).sum().item()
          
    val_loss /= len(val_loader.dataset)
    test_accuracy = 100.0 * correct / len(val_loader.dataset)
    print('test accuracy is : ', test_accuracy)
    print(f'epoch: {epoch+1}, train loss:{train_loss:.4f}, val loss:{val_loss:.4f}' )
    print('var loss: ', val_loss, 'best var loss: ', best_val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save( model, 'best_model.pt') # 개선된모델저장
        
    else:
        patience_counter +=1
        print( f'no improvement:{patience_counter/patience}')
        if patience_counter >= patience:
            print( 'early stop ')
            break
    
    

12544
0 train
1 train
2 train
3 train
4 train
5 train
6 train
7 train
8 train
9 train
10 train
11 train
12 train
13 train
14 train
15 train
16 train
17 train
18 train
19 train
20 train
21 train
22 train
23 train
24 train
25 train
26 train
27 train
28 train
29 train
train loss is  2.518920393784841
0 evaluate
1 evaluate
2 evaluate
3 evaluate
4 evaluate
5 evaluate
6 evaluate
7 evaluate
test accuracy is :  47.93333333333333
epoch: 1, train loss:2.5189, val loss:1.4597
var loss:  1.4597058817545574 best var loss:  inf
0 train
1 train
2 train
3 train
4 train
5 train
6 train
7 train
8 train
9 train
10 train
11 train
12 train
13 train
14 train
15 train
16 train
17 train
18 train
19 train
20 train
21 train
22 train
23 train
24 train
25 train
26 train
27 train
28 train
29 train
train loss is  1.2225739041964212
0 evaluate
1 evaluate
2 evaluate
3 evaluate
4 evaluate
5 evaluate
6 evaluate
7 evaluate
test accuracy is :  60.65833333333333
epoch: 2, train loss:1.2226, val loss:1.0300
var loss:  1.02

In [91]:
for i, (batch_x, batch_y) in enumerate(train_loader):
    print(batch_x[0].reshape(1, 1, 28, 28).shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])


In [99]:
model = torch.load('best_model.pt').to(DEVICE)
pred= model(x_test[0].reshape(1, 1, 28, 28).to(DEVICE))
idx = pred.argmax(1, keepdim=True)
print('예측결과', idx.item() == y_test[0], '\nx_test의 예측값:', idx.item(), '실제값: ', y_test[0].item())

예측결과 tensor(True) 
x_test의 예측값: 9 실제값:  9
